In [55]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import re
import numpy as np

In [69]:
url = 'https://www.indeed.com/jobs?q=Data+Science&l=Los+Angeles%2C+CA&vjk=74cadd03cb0dd918'
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(url)

In [4]:
container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")

In [5]:
job_one = container.find_element(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")

In [6]:
#first_title = job_one.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
#first_company = job_one.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
#first_rating = job_one.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
#job_one.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[2]
#n = job_one.find_element(By.XPATH, ".//table[contains(@role, 'presentation')]").text.split("\n")

In [7]:
job_one.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text

'DataAnnotation is committed to creating quality AI. Join our team to help train AI chatbots while gaining the flexibility of remote work and choosing your own…\nEmployer\nActive Today'

In [8]:
#job_one.find_element(By.XPATH, "//span[contains(@data-testid, 'myJobsStateDate')]").text.split("\n")[1]
job_one.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")

'Active Today'

In [9]:
job_one.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")

['DataAnnotation', '4.5', 'Remote in Los Angeles, CA']

In [10]:
pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text
pagination.find_elements(By.XPATH, ".//li[contains(@class, 'css227srf')]")


[]

In [11]:
container.find_elements(By.XPATH, ".//li[contains(@class, 'css-5lfssm')]")

[<selenium.webdriver.remote.webelement.WebElement (session="fa865dad2c59f04f6c356f447ee8fc91", element="f.FA88FE06569919D6E3C606D758D18725.d.E8C8FB5F0AAEF5DE0C3DE191C3751448.e.500")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa865dad2c59f04f6c356f447ee8fc91", element="f.FA88FE06569919D6E3C606D758D18725.d.E8C8FB5F0AAEF5DE0C3DE191C3751448.e.577")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa865dad2c59f04f6c356f447ee8fc91", element="f.FA88FE06569919D6E3C606D758D18725.d.E8C8FB5F0AAEF5DE0C3DE191C3751448.e.635")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa865dad2c59f04f6c356f447ee8fc91", element="f.FA88FE06569919D6E3C606D758D18725.d.E8C8FB5F0AAEF5DE0C3DE191C3751448.e.704")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa865dad2c59f04f6c356f447ee8fc91", element="f.FA88FE06569919D6E3C606D758D18725.d.E8C8FB5F0AAEF5DE0C3DE191C3751448.e.756")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fa865dad2c59f04f6c35

In [12]:
# try getting pay amount
string = job_one.find_element(By.XPATH, ".//div[contains(@class, 'salary-snippet-container')]").text
listy = string.split()
for element in listy:
    if "$" in element:
        pay = element

print(pay.strip("$"))


40


In [70]:
job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1

pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text

while current_page <= int(last_page):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        try:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        except:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        when_posted= job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")
        description = job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[0]
        try:
            string = job.find_element(By.XPATH, ".//div[contains(@class, 'salary-snippet-container')]").text
            listy = string.split()
            for element in listy:
                if "$" in element:
                    pay = element
        except:
            pay = "NaN"
        job_titles.append(title)
        job_pays.append(pay)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    current_page += 1
    try:
        next = driver.find_element(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-next')]")
        next.click()
    except:
        pass

In [43]:
# goes to 5 pages
job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1

pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text

while current_page <= int(last_page):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        try:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        except:
            location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        when_posted= job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[-1].strip("More...")
        description = job.find_element(By.XPATH, ".//tr[contains(@class, 'underShelfFooter')]").text.split("\n")[0]
        job_titles.append(title)
        job_pays.append(pay)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    current_page += 1
    try:
        next = driver.find_element(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-next')]")
        next.click()
    except:
        pass

In [21]:
job_titles = []
company_names = []
company_ratings = []
company_locations = []
job_pays = []
descriptions = []
posted = []
current_page = 1

pagination = driver.find_element(By.XPATH, ".//nav[contains(@role, 'navigation')]")
last_page = pagination.find_elements(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-5')]")[-1].text

while current_page <= int(last_page):
    container = driver.find_element(By.XPATH, ".//div[contains(@id, 'mosaic-jobResults')]")
    all_jobs = container.find_elements(By.XPATH, ".//div[contains(@class, 'job_seen_beacon')]")
    for job in all_jobs:
        title = job.find_element(By.XPATH, ".//h2[contains(@class, 'jobTitle')]").text
        company = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[0]
        location = job.find_element(By.XPATH, ".//div[contains(@class, 'company_location')]").text.split("\n")[1]
        when_posted= job.find_element(By.XPATH, "//span[contains(@data-testid, 'myJobsStateDate')]").text.split("\n")[1]
        description = job.find_element(By.XPATH, "//div[contains(@class, 'css-9446fg')]").text
        job_titles.append(title)
        company_names.append(company)
        descriptions.append(description)
        company_locations.append(location)
        posted.append(when_posted)
    current_page += 1
    try:
        next = driver.find_element(By.XPATH, ".//a[contains(@data-testid, 'pagination-page-next')]")
        next.click()
    except:
        pass



IndexError: list index out of range

In [71]:
#separate into city and state
cities = []
states = []

# Parse location data
for location in company_locations:
    parts = location.split(', ')
    city = parts[0]
    state = parts[1] if len(parts) > 1 else None
    cities.append(city)
    states.append(state[:2])

TypeError: 'NoneType' object is not subscriptable

In [56]:
#make salaries integers
salaries = []
for salary in job_pays:
    if salary == "NaN" or salary == "":
        salaries.append(np.nan)
    else:
        salaries.append(int(salary.replace('$', '').replace(',', '')))

In [72]:

job_info = pd.DataFrame({"Title": job_titles,
              "Company": company_names,
              "Location": company_locations,
              "Job Description": descriptions,
              "When Posted": posted,
              "Pay": salaries,
              "City":cities,
               "State": states})
job_info

ValueError: All arrays must be of the same length

In [66]:
#try getting hybrid or remote status
def is_hybrid_remote(location):
    return 'Hybrid' in location or 'Remote' in location

# Apply the function to create a new column 'Is_Hybrid_Remote'
job_info['Is_Hybrid_Remote'] = job_info['Location'].apply(is_hybrid_remote)
job_info

,Title,Company,Location,Job Description,When Posted,Pay,City,State,Is_Hybrid_Remote
0,Product Data Analyst (Gaming),TapBlaze,"Los Angeles, CA 90025",Data Integrity: Ensure data accuracy and integ...,Posted 30+ days ago·,NaN,Los Angeles,CA,False
1,Data Scientist IV,OrangePeople,"Bell, CA 90201","Experience finding, cleaning, and manipulating...",Active 4 days ago·,NaN,Bell,CA,False
2,Data Scientist- CalAIM,Heluna Health,"Los Angeles, CA 90014",Coordinates and contributes to complex project...,Posted 30+ days ago·,NaN,Los Angeles,CA,False
3,Data Scientist,Core Digital Media,"Hybrid remote in Los Angeles, CA",Do data explorations daily to uncover opportun...,Posted 30+ days ago·,NaN,Hybrid remote in Los Angeles,CA,True
4,Data Science Instructor,UCLA Extension,"Los Angeles, CA","Minimum MS degree in computer science, statist...",Posted 30+ days ago·,NaN,Los Angeles,CA,False
...,...,...,...,...,...,...,...,...,...
70,DATA SCIENTIST,US Department of the Air Force - Agency Wide,"Los Angeles, CA","Degree: Mathematics, statistics, computer scie...",Posted 30+ days ago·,NaN,Los Angeles,CA,False
71,Senior Live Product Data Analyst,TikTok,"Hybrid remote in Los Angeles, CA","Bachelor degree or above, major in mathematics...",Posted 30+ days ago·,220611.0,Hybrid remote in Los Angeles,CA,True
72,"Manager, Data Science",Disney Direct to Consumer,"Santa Monica, CA",Experience fully integrating data science solu...,Posted 30+ days ago·,200200.0,Santa Monica,CA,False
73,Data Engineer (L4) - Content Knowledge Graph,Netflix,"Los Angeles, CA",Obsessed data quality: build reliable and trus...,Posted 30+ days ago·,NaN,Los Angeles,CA,False


In [68]:
#job_info.to_csv("more_info.csv")

In [67]:
driver.quit()